<a href="https://colab.research.google.com/github/shanksghub/bitcoin/blob/master/LTSM%20Traceback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.recurrent import GRU
from keras.layers.core import Dense, Dropout
url = 'https://raw.githubusercontent.com/shanksghub/bitcoin/master/convertcsv.csv'
df = pd.read_csv(url, error_bad_lines=False)
df.head()

a= df.drop(columns=['Date'])

In [47]:
df = df[['Close']]


dataset = df.values
dataset = dataset.astype('float32')

dataset


array([[11358.662 ],
       [12156.513 ],
       [12573.8125],
       ...,
       [  139.    ],
       [  144.54  ],
       [  134.21  ]], dtype=float32)

In [0]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)


In [49]:
dataset


array([[0.58110297],
       [0.62216794],
       [0.6436462 ],
       ...,
       [0.00363215],
       [0.00391729],
       [0.00338561]], dtype=float32)

In [50]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
print(len(train), len(test))

1515 747


In [0]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
  dataX, dataY = [], []
  for i in range(len(dataset)-look_back-1):
    a = dataset[i:(i+look_back), 0]
    dataX.append(a)
    dataY.append(dataset[i + look_back, 0])
  return np.array(dataX), np.array(dataY)

In [0]:
look_back = 10
trainX, trainY = create_dataset(train, look_back=look_back)
testX, testY = create_dataset(test, look_back=look_back)

In [53]:
trainX


array([[0.58110297, 0.62216794, 0.6436462 , ..., 0.57373124, 0.6121189 ,
        0.5524352 ],
       [0.62216794, 0.6436462 , 0.62883043, ..., 0.6121189 , 0.5524352 ,
        0.5411869 ],
       [0.6436462 , 0.62883043, 0.5858477 , ..., 0.5524352 , 0.5411869 ,
        0.55323184],
       ...,
       [0.00868111, 0.00869717, 0.00869073, ..., 0.00884849, 0.00859093,
        0.00852269],
       [0.00869717, 0.00869073, 0.00868214, ..., 0.00859093, 0.00852269,
        0.00841609],
       [0.00869073, 0.00868214, 0.00868183, ..., 0.00852269, 0.00841609,
        0.00847688]], dtype=float32)

In [54]:
trainY

array([0.5411869 , 0.55323184, 0.61202115, ..., 0.00841609, 0.00847688,
       0.00866598], dtype=float32)

In [0]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [58]:
model = Sequential()

model.add(GRU(units=512,
              return_sequences=True,
              input_shape=(1, 10)))
model.add(Dropout(0.2))
model.add(GRU(units=256))
model.add(Dropout(0.2))

model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=256, verbose=2)

ValueError: ignored

In [59]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)


W0801 15:30:04.884043 140683295401856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [61]:
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:, 0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:, 0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 5225.68 RMSE
Test Score: 397.75 RMSE


In [62]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict) + look_back, :] = trainPredict

ValueError: ignored

In [0]:
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict) + (look_back * 2) + 1:len(dataset) - 1, :] = testPredict

In [0]:
plt.plot(df['Close'], label='Actual')
plt.plot(pd.DataFrame(trainPredictPlot, columns=["close"], index=df.index).close, label='Training')
plt.plot(pd.DataFrame(testPredictPlot, columns=["close"], index=df.index).close, label='Testing')
plt.legend(loc='best')
plt.show()